In [1]:
import pandas as pd
import numpy as np
import glob
import os
from sklearn.preprocessing import OneHotEncoder
from sklearn.utils import class_weight
import pickle
import random

Category = "Country"
Africa = ['DZ', 'EG', 'MA', 'ZA', 'TN']
Asia = ['BH', 'HK', 'IN', 'ID', 'IL', 'JP', 'JO', 'KW', 
        'LB', 'MY', 'OM', 'PS', 'PH', 'QA', 'SA', 'SG', 
        'TW', 'TH', 'AE', 'VN']
Europe = ['AD', 'AT', 'BE', 'BG', 'CY', 'CZ', 'DK', 'EE', 
          'FI', 'FR', 'DE', 'GR', 'HU', 'IS', 'IE', 'IT', 
          'LV', 'LI', 'LT', 'LU', 'MT', 'MC', 'NL', 'NO', 
          'PL', 'PT', 'RO', 'SK', 'ES', 'SE', 'CH', 'TR', 'GB']
NorthAmerica = ['CA', 'CR', 'DO', 'SV', 'GT', 'HN', 'MX', 'NI', 'PA', 'US']
SouthAmerica = ['AR', 'BO', 'BR', 'CL', 'CO', 'EC', 'PY', 'PE', 'UY']
Oceania = ['AU', 'NZ']

countriesOfInterest = Africa + Asia + Europe + NorthAmerica + SouthAmerica + Oceania
#countriesOfInterest = ["HK", "JP", 'ZA', 'TN', 'TR', 'GB', 'MX', 'US', 'CO', 'EC', 'AU', 'NZ']

w_length = 200

### Consolidate and pickle country files

In [2]:
for country in countriesOfInterest:
    if len(glob.glob("Raw Track Data\\" + country + ".p")) == 0 :
        hold = pd.DataFrame()
    else:
        hold = pickle.load( open( "Raw Track Data\\" + country + ".p", "rb" ) )
    print(country, hold.shape)
    countryTracks = glob.glob("Raw Track Data\\" + country + "*.csv")
    print(country, len(countryTracks))
    for file in countryTracks:
        new = pd.read_csv(file)
        new["Country"] = file[15:17]
        new["Year"] = file[18:22]
        new["Playlist"] = file[23:-4]
        hold = hold.append(new)
        pickle.dump( hold, open( "Raw Track Data\\" + country + ".p", "wb" ) )
        os.remove(file)
    hold = hold.drop_duplicates(["track_id", "start"])
    pickle.dump( hold, open( "Raw Track Data\\" + country + ".p", "wb" ) )

DZ (36299, 34)
DZ 34
EG (151231, 34)
EG 32
MA (79688, 34)
MA 34
ZA (1492095, 34)
ZA 9
TN (153437, 34)
TN 17
BH (16939, 34)
BH 38
HK (500127, 34)
HK 11
IN (1106, 34)
IN 18
ID (1168135, 34)
ID 23
IL (903602, 34)
IL 15
JP (1155831, 34)
JP 6
JO (31052, 34)
JO 32
KW (10846, 34)
KW 40
LB (674, 34)
LB 26
MY (501083, 34)
MY 27
OM (13487, 34)
OM 40
PS (14401, 34)
PS 26
PH (740166, 34)
PH 19
QA (7547, 34)
QA 40
SA (1552, 34)
SA 40
SG (468473, 34)
SG 25
TW (1647999, 34)
TW 32
TH (919928, 34)
TH 25
AE (626723, 34)
AE 10
VN (941608, 34)
VN 7
AD (8043, 34)
AD 19
AT (116183, 34)
AT 19
BE (575009, 34)
BE 19
BG (7768, 34)
BG 19
CY (0, 34)
CY 12
CZ (51422, 34)
CZ 11
DK (1670583, 34)
DK 10
EE (8195, 34)
EE 9
FI (1366905, 34)
FI 14
FR (1152809, 34)
FR 10
DE (277802, 34)
DE 9
GR (0, 34)
GR 9
HU (289422, 34)
HU 9
IS (474975, 34)
IS 10
IE (984620, 34)
IE 15
IT (619032, 34)
IT 17
LV (1966, 34)
LV 17
LI (0, 34)
LI 17
LT (0, 34)
LT 17
LU (1402, 34)
LU 17
MT (0, 34)
MT 17
MC (3215, 34)
MC 17
NL (882289, 34)
NL 2

In [3]:
hold

,start,duration,confidence,loudness_start,loudness_max_time,loudness_max,p1,p2,p3,p4,...,t7,t8,t9,t10,t11,t12,track_id,Country,Year,Playlist
681,0.00000,0.49914,0.000,-60.000,0.00000,-60.000,0.019,0.040,0.045,0.024,...,14.833,5.359,-27.228,0.973,-10.640,-7.228,spotify:track:7imVDfvNz2H4szraikF3Hd,NZ,2015,37i9dQZF1DX9TriA5Rm2k8
682,0.49914,0.22644,1.000,-60.000,0.03370,-10.164,0.646,0.420,0.522,0.560,...,-20.233,-14.167,-6.882,34.476,41.585,7.220,spotify:track:7imVDfvNz2H4szraikF3Hd,NZ,2015,37i9dQZF1DX9TriA5Rm2k8
683,0.72558,0.12181,0.381,-31.803,0.00782,-26.013,0.940,0.858,0.439,0.460,...,16.783,-18.108,13.722,32.195,-23.695,-3.539,spotify:track:7imVDfvNz2H4szraikF3Hd,NZ,2015,37i9dQZF1DX9TriA5Rm2k8
684,0.84739,0.60340,0.952,-26.892,0.18692,-14.485,0.073,0.149,0.097,0.237,...,64.849,-42.428,21.399,50.362,-43.406,-18.990,spotify:track:7imVDfvNz2H4szraikF3Hd,NZ,2015,37i9dQZF1DX9TriA5Rm2k8
685,1.45079,0.20980,0.775,-21.574,0.04262,-13.192,0.296,0.377,0.302,0.450,...,38.661,-2.748,52.698,13.987,-17.991,-5.808,spotify:track:7imVDfvNz2H4szraikF3Hd,NZ,2015,37i9dQZF1DX9TriA5Rm2k8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55197,150.29098,0.80113,0.770,-22.333,0.06393,-14.336,0.759,0.550,0.569,0.753,...,-14.465,5.575,-10.674,24.237,4.324,9.846,spotify:track:6iTHsvN6Mi2xaQ0igLNvTD,NZ,2013,37i9dQZF1DXd5DCuoVuFY3
55198,151.09211,0.56880,0.493,-18.990,0.05247,-13.539,1.000,0.438,0.370,0.325,...,9.500,-19.003,-12.108,35.121,-17.146,14.205,spotify:track:6iTHsvN6Mi2xaQ0igLNvTD,NZ,2013,37i9dQZF1DXd5DCuoVuFY3
55199,151.66091,0.54032,0.523,-21.277,0.01967,-15.467,0.010,0.006,0.005,0.004,...,8.364,2.038,-2.469,14.671,-7.272,13.971,spotify:track:6iTHsvN6Mi2xaQ0igLNvTD,NZ,2013,37i9dQZF1DXd5DCuoVuFY3
55200,152.20122,0.50481,0.578,-28.868,0.03622,-22.509,0.257,0.221,0.110,0.061,...,3.100,2.415,9.850,4.093,-7.481,0.935,spotify:track:6iTHsvN6Mi2xaQ0igLNvTD,NZ,2013,37i9dQZF1DXd5DCuoVuFY3


### Remove non-unique tracks

In [4]:
allTracks = pd.DataFrame()
for country in countriesOfInterest:
    hold = pickle.load( open( "Raw Track Data\\" + country + ".p", "rb" ) )
    hold = hold[["Country", "Year", "track_id", "Playlist"]]
    allTracks = allTracks.append(hold)
allTracks

,Country,Year,track_id,Playlist
19557,DZ,2016,spotify:track:1yxafhHFJMYkA8oSlLz68Q,37i9dQZF1DWVSIz2AGspV4
19558,DZ,2016,spotify:track:1yxafhHFJMYkA8oSlLz68Q,37i9dQZF1DWVSIz2AGspV4
19559,DZ,2016,spotify:track:1yxafhHFJMYkA8oSlLz68Q,37i9dQZF1DWVSIz2AGspV4
19560,DZ,2016,spotify:track:1yxafhHFJMYkA8oSlLz68Q,37i9dQZF1DWVSIz2AGspV4
19561,DZ,2016,spotify:track:1yxafhHFJMYkA8oSlLz68Q,37i9dQZF1DWVSIz2AGspV4
...,...,...,...,...
55197,NZ,2013,spotify:track:6iTHsvN6Mi2xaQ0igLNvTD,37i9dQZF1DXd5DCuoVuFY3
55198,NZ,2013,spotify:track:6iTHsvN6Mi2xaQ0igLNvTD,37i9dQZF1DXd5DCuoVuFY3
55199,NZ,2013,spotify:track:6iTHsvN6Mi2xaQ0igLNvTD,37i9dQZF1DXd5DCuoVuFY3
55200,NZ,2013,spotify:track:6iTHsvN6Mi2xaQ0igLNvTD,37i9dQZF1DXd5DCuoVuFY3


In [5]:
hold = allTracks.groupby("track_id").nunique()
print(hold.shape[0], "Unique tracks per country")
keep = hold[hold.Country==1].index
allTracks = allTracks[allTracks.track_id.isin(keep)]
print(len(keep), "Unique tracks across countries")

57842 Unique tracks per country
46100 Unique tracks across countries


In [6]:
for country in countriesOfInterest:
    hold = pickle.load( open( "Raw Track Data\\" + country + ".p", "rb" ) )
    hold = hold[hold.track_id.isin(keep)]
    pickle.dump( hold, open( "Raw Track Data\\" + country + ".p", "wb" ) )

### Create training, testing, and validation datasets

In [7]:
for country in countriesOfInterest:
    hold = pickle.load( open( "Raw Track Data\\" + country + ".p", "rb" ) )
    UniqueTracks = pd.unique(hold.track_id)
    testTracks = np.random.choice(UniqueTracks, int(len(UniqueTracks) * .2), replace = False)
    test = hold.loc[hold.track_id.isin(testTracks)]
    hold = hold.loc[~hold.track_id.isin(testTracks)]
    UniqueTracks = pd.unique(hold.track_id)
    testTracks = np.random.choice(UniqueTracks, int(len(UniqueTracks) * .2), replace = False)
    val = hold.loc[hold.track_id.isin(testTracks)]
    train = hold.loc[~hold.track_id.isin(testTracks)]
    pickle.dump( train, open( "Raw Track Data\\" + country + "_train.p", "wb" ) )
    pickle.dump( test, open( "Raw Track Data\\" + country + "_test.p", "wb" ) )
    pickle.dump( val, open( "Raw Track Data\\" + country + "_val.p", "wb" ) )